In [91]:
import math
import xlrd
import xlwt
import numpy as np
import time
import matplotlib.pyplot as plt
from math import *

nameofop = input()
mfuture = xlrd.open_workbook(u'Zoom_2.xls').sheets()[0]
moption = xlrd.open_workbook(u'Moption price_'+nameofop+'.xlsx').sheets()[0]
dayvdata = xlrd.open_workbook(u'dayvolmonths.xlsx').sheets()[0]
n1 = 4
n2 = 6
taryield = 0.01


2700


In [92]:
def phi(x):
    #'Cumulative distribution function for the standard normal distribution'
    return (1.0 + erf(x / sqrt(2.0))) / 2.0


def bsform(s,x,r,sigma,tau):
    d1 = (np.log(s/x)+(r+sigma*sigma/2)*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*math.sqrt(tau)
    return s*phi(d1)-x*math.exp(-r*tau)*phi(d2)

def calc_impliedvol(st, en, op_price, fut_price, tau, sig30, date):
    if en - st < 0.0000000001:
        return st
    else:
        mid = (st+en)/2
        pst = bsform(fut_price, int(nameofop), 0.025, st, tau) - op_price
        pen = bsform(fut_price, int(nameofop), 0.025, en, tau) - op_price
        pmid = bsform(fut_price, int(nameofop), 0.025, mid, tau) - op_price
        if pst*pmid <= 0:
            return calc_impliedvol(st, mid, op_price, fut_price, tau, sig30, date)
        if pen*pmid <= 0:
            return calc_impliedvol(mid, en, op_price, fut_price, tau, sig30, date)
        if pst*pmid > 0 and pen*pmid > 0:
            print("a replace occured"+str(st)+': '+str(pst+op_price)+' —— '+str(en)+': '+str(pen+op_price))
            print("op_price: "+ str(op_price))
            return sig30


def my_std(inplist, days):
    return [0 for i in range(days)]+[np.std(inplist[i:i+days], ddof = 1) for i in range(len(inplist)-days+1)]

def my_norm(inp):
    return np.exp(-inp*inp/2)/math.sqrt(2*math.pi)


def calc_vega(price_fut,vol,tau):
    d1 = (np.log(price_fut/int(nameofop))+(0.025+vol*vol/2)*tau)/(vol*np.sqrt(tau))
    vega = np.sqrt(tau)*price_fut*my_norm(d1)
    return vega


In [93]:
price_op = np.array(moption.col_values(1)[1:])
price_fut = np.array(mfuture.col_values(1)[1:])
t_date = np.array(mfuture.col_values(0)[1:])

price_st = np.array(dayvdata.col_values(1)[1:])
price_low = np.array(dayvdata.col_values(2)[1:])
tau = np.array(dayvdata.col_values(4)[1:])/365
day_vol = np.log(price_st/price_low)*np.log(price_st/price_low)/(4*np.log(2))

print(tau)
vol30 = np.array(mfuture.col_values(4)[1:])
impvol = np.array([calc_impliedvol(0.000001,0.5, price_op[i], price_fut[i], tau[i], vol30[i], t_date[i]) for i in range(len(price_op))] )
print(impvol)
vega = calc_vega(price_fut,impvol,tau)
d1 = (np.log(price_st/int(nameofop))+(0.025+impvol*impvol/2)*tau)/(impvol*np.sqrt(tau))
delta = [phi(i) for i in d1]
sigma = vega*day_vol



[0.33972603 0.3369863  0.33424658 0.3260274  0.32328767 0.32054795
 0.31780822 0.31506849 0.30684932 0.30410959 0.30136986 0.29863014
 0.29589041 0.28767123 0.28493151 0.28219178 0.27945205 0.27671233
 0.26575342 0.2630137  0.26027397 0.25753425 0.24931507 0.24657534
 0.24383562 0.24109589 0.23835616 0.23013699 0.22739726 0.22465753
 0.22191781 0.21917808 0.2109589  0.20821918 0.20547945 0.20273973
 0.2        0.18630137 0.18356164 0.18082192 0.17260274 0.16986301
 0.16712329 0.16438356 0.16164384 0.15342466 0.15068493 0.14794521
 0.14520548 0.14246575 0.13424658 0.13150685 0.12876712 0.1260274
 0.12328767 0.11506849 0.11232877 0.10958904 0.10684932 0.10410959
 0.09589041 0.09315068 0.09041096 0.08767123 0.08493151 0.07671233
 0.0739726  0.07123288 0.06849315 0.06575342 0.05753425 0.05479452
 0.05205479 0.04931507 0.04657534 0.03835616 0.03561644 0.03287671
 0.03013699 0.02739726 0.01917808]
a replace occured1e-06: 234.98853657528343 —— 0.5: 297.8335092191928
op_price: 231.5
a replace 

In [63]:
pos = 0
open_pos = 0
cyield = 0.00
tyield = [0]

sigv1 =[99999 for i in range(0,n2)] + [np.sum(sigma[i-n1:i])/np.sum(vega[i-n1:i]) for i in range(n2,len(sigma))]
sigv2=[99999 for i in range(0,n2)] + [np.sum(sigma[i-n2:i])/np.sum(vega[i-n1:i]) for i in range(n2,len(sigma))]

for i in range(n2, len(sigma)):
    if (cyield > taryield) or (sigv1[i]>sigv1[i-1]):
        cyield = 0
        pos = 0
    if pos == 0 and sigv1[i]<sigv2[i]:
        open_pos = 1
    if pos == 0 and open_pos == 1:
        open_pos = 0
        pos = 1
        cyield += math.log(-price_op[i]+delta[i]*price_fut[i])- math.log(-price_op[i-1]+delta[i]*price_fut[i-1])
        tyield.append(tyield[len(tyield)-1]+(-price_op[i]+delta[i]*price_fut[i])-(-price_op[i-1]+delta[i]*price_st[i]))
        continue
    if pos == 1:
        cyield += math.log(-price_op[i]+delta[i]*price_fut[i])- math.log(-price_op[i-1]+delta[i]*price_fut[i-1])
        tyield.append(tyield[len(tyield)-1]+(-price_op[i]+delta[i]*price_fut[i])-(-price_op[i-1]+delta[i]*price_st[i]))
    else:
        tyield.append(tyield[len(tyield)-1])

print(tyield)
    
    


[0, 0.9758072047688984, -0.54583089277412, -4.5934743524449, -4.210467953508669, 9.55415711416208, 23.230908305005414, 21.336406059068395, 21.383976216349765, 26.383976216349765, 15.585803906124966, 10.470255966853983, 10.882056867756546, 0.31975399450084296, 7.681468155579296, 10.181468155579296, 8.252028797373441, -11.045309976052522, -10.221492019087691, -6.182216192069518, -4.488533745228324, -5.830811989496397, -6.566101161147799, -10.780490484948587, -16.358136259639195, -22.20220555158221, -14.389719356310252, -13.312721513602128, -14.633064188849403, -15.934946554010821, -20.523026298403465, -25.714115159632456, -25.614416280827527, -23.54647582651785, -26.639207191106493, -25.46532325645319, -27.171543840036065, -26.98337234383075, -32.12936004687526, -37.20960695117492, -40.6140743183546, -36.4575285765477, -34.649149102865294, -33.03197106924654, -27.807547168789142, -29.98504324952728, -27.30786017734431, -21.95976918063434, -23.498647447452186, -21.336231267809353, -22.704

In [64]:
output = xlwt.Workbook(encoding = 'ascii')
sheet1 = output.add_sheet('Sheet1',cell_overwrite_ok= True)
for i in range(0, len(tyield)):
    sheet1.write(i+6,1,tyield[i])
for i in range(0, len(vega)):
    sheet1.write(i+1,2,vega[i])
    sheet1.write(i+1,3,delta[i])
    sheet1.write(i+1,4,price_fut[i])
    sheet1.write(i+1,5,price_op[i])
    sheet1.write(i+1,6,day_vol[i])
    sheet1.write(i+1,7,sigv1[i])
    sheet1.write(i+1,8,sigv2[i])
    sheet1.write(i+1,9,impvol[i])
    sheet1.write(i+1,10,vol30[i])
output.save('fm_c'+nameofop+'immm.xls')